In [2]:
import pandas as pd


df = pd.read_csv("data/2021-01-01_2025-07-25_hrrr_env_final_v4.csv")
df.columns

Index(['gauge_idx', 'start_datetime_utc', 'end_datetime_utc', 'gauge_acc_in',
       'mrms_q3evap_qpe', 'lat', 'lon', '700mb_UGRD', '850mb_DPT', '850mb_TMP',
       '850mb_HGT', '2m_above_ground_TMP', '925mb_DPT',
       'entire_atmosphere_single_layer_PWAT', '500mb_TMP', '700mb_DPT',
       '700mb_TMP', '500mb_DPT', '700mb_HGT', '850mb_UGRD', '700mb_VGRD',
       'highest_tropospheric_freezing_level_HGT', 'surface_PRES',
       'level_of_adiabatic_condensation_from_sfc_HGT', '850mb_VGRD',
       '2m_above_ground_DPT', 'surface_theta_e',
       'lowest_100mb_mean_mixing_ratio', '925mb_700mb_DPT', '850mb_700mb_DPT',
       '850mb_500mb_DPT', 'LCL_LFC_RH', '0-3km_RH', '0-5km_RH', 'DCAPE',
       'low_level_lapse_rate', '3hr_lapse_rate_change', 'sfc_850_pw',
       'sfc_700_pw', '850_700_mean_wind', 'CWCD', 'LCL_height'],
      dtype='object')

In [33]:
# — LCL —-
from metpy.calc import lcl, pressure_to_height_std
from metpy.units import units
from tqdm import tqdm

tqdm.pandas()


pres, temp, dewp   = df["surface_PRES"].values, df["2m_above_ground_TMP"].values, df["2m_above_ground_DPT"].values

pres *= units.Pa
temp *= units.K
dewp *= units.K

lcl_pres, lcl_temp = lcl(pres, temp, dewp)
lcl_height_arr     = pressure_to_height_std(lcl_pres)
lcl_height_arr

# lcl(943 * units.hPa, 33 * units.degC, 28 * units.degC)
# (<Quantity(877.033549, 'hectopascal')>, <Quantity(26.7591908, 'degree_Celsius')>)

Magnitude,[2.99047671079993 3.020788278810226 3.275219125386256 ... 4.475853794876949 4.475853794876949 4.475853794876949]
Units,kilometer


In [42]:
df.columns

Index(['gauge_idx', 'start_datetime_utc', 'end_datetime_utc', 'gauge_acc_in',
       'mrms_q3evap_qpe', 'lat', 'lon', '700mb_UGRD', '850mb_DPT', '850mb_TMP',
       '850mb_HGT', '2m_above_ground_TMP', '925mb_DPT',
       'entire_atmosphere_single_layer_PWAT', '500mb_TMP', '700mb_DPT',
       '700mb_TMP', '500mb_DPT', '700mb_HGT', '850mb_UGRD', '700mb_VGRD',
       'highest_tropospheric_freezing_level_HGT', 'surface_PRES',
       'level_of_adiabatic_condensation_from_sfc_HGT', '850mb_VGRD',
       '2m_above_ground_DPT', 'surface_theta_e',
       'lowest_100mb_mean_mixing_ratio', '925mb_700mb_DPT', '850mb_700mb_DPT',
       '850mb_500mb_DPT', 'LCL_LFC_RH', '0-3km_RH', '0-5km_RH', 'DCAPE',
       'low_level_lapse_rate', '3hr_lapse_rate_change', 'sfc_850_pw',
       'sfc_700_pw', '850_700_mean_wind', 'CWCD', 'LCL_height'],
      dtype='object')

In [46]:
VAR = "DCAPE"

df[VAR].min(), df[VAR].max()

(np.float64(0.0), np.float64(198.22163750162747))

In [ ]:
import numpy as np
from metpy.units import units
from metpy.calc import downdraft_cape

# Pre-create unit objects to avoid repeated creation
_hPa = units.hPa
_K = units.K


def calc_dcape(
    surface_pres_pa,
    temp_2m_k, dewpoint_2m_k,
    temp_850_k, dewpoint_850_k,
    temp_700_k, dewpoint_700_k,
    temp_500_k, dewpoint_500_k
):
    """
    Calculate DCAPE from sparse pressure level data.
    Returns:
        DCAPE in J/kg, or np.nan if calculation fails
    """
    try:
        sfc_p_hpa = surface_pres_pa * 0.01  # Multiply is faster than divide
        
        # Build profiles - pre-sized lists avoid repeated growth
        p_list = [sfc_p_hpa]
        t_list = [temp_2m_k]
        td_list = [dewpoint_2m_k]
        
        # Unrolled loop - avoids tuple unpacking overhead
        if 850 < sfc_p_hpa:
            p_list.append(850)
            t_list.append(temp_850_k)
            td_list.append(dewpoint_850_k)
        
        if 700 < sfc_p_hpa:
            p_list.append(700)
            t_list.append(temp_700_k)
            td_list.append(dewpoint_700_k)
        
        if 500 < sfc_p_hpa:
            p_list.append(500)
            t_list.append(temp_500_k)
            td_list.append(dewpoint_500_k)
        
        if len(p_list) < 2:
            return np.nan
        
        # Use pre-created unit objects
        p = np.array(p_list) * _hPa
        T = np.array(t_list) * _K
        Td = np.array(td_list) * _K
        
        dcape, _, _ = downdraft_cape(p, T, Td)
        return dcape.magnitude
        
    except Exception:
        return np.nan  # Removed slow print statement


def proc_row_dcape(row):
    return calc_dcape(
        row["surface_PRES"],
        row["2m_above_ground_TMP"],
        row["2m_above_ground_DPT"],
        row["850mb_TMP"],
        row["850mb_DPT"],
        row["700mb_TMP"],
        row["700mb_DPT"],
        row["500mb_TMP"],
        row["500mb_DPT"],
    )


# Move tqdm.pandas() call to execution time
from tqdm import tqdm
tqdm.pandas()

dcape = df.progress_apply(proc_row_dcape, axis=1)

In [ ]:
# — LFC —
from metpy.calc import dewpoint_from_relative_humidity, lfc
from metpy.units import units


# pressure
p = [1008., 1000., 950., 900., 850., 800., 750., 700., 650., 600.,
     550., 500., 450., 400., 350., 300., 250., 200.,
     175., 150., 125., 100., 80., 70., 60., 50.,
     40., 30., 25., 20.] * units.hPa
# temperature
T = [29.3, 28.1, 23.5, 20.9, 18.4, 15.9, 13.1, 10.1, 6.7, 3.1,
     -0.5, -4.5, -9.0, -14.8, -21.5, -29.7, -40.0, -52.4,
     -59.2, -66.5, -74.1, -78.5, -76.0, -71.6, -66.7, -61.3,
     -56.3, -51.7, -50.7, -47.5] * units.degC
# relative humidity
rh = [.85, .65, .36, .39, .82, .72, .75, .86, .65, .22, .52,
      .66, .64, .20, .05, .75, .76, .45, .25, .48, .76, .88,
      .56, .88, .39, .67, .15, .04, .94, .35] * units.dimensionless
# calculate dewpoint
Td = dewpoint_from_relative_humidity(T, rh)
# calculate LFC
lfc(p, T, Td)


# fetch historic sounding data
import sounderpy as spy
spy.get_obs_data(...)

$\theta e$ equation:
- $\theta e = \theta * \exp(\frac{LV * r}{Cp_d * T})$

In [ ]:
VAR = "lowest_100mb_mean_mixing_ratio"
df[VAR].min(), df[VAR].max()

(np.float64(-1726.270045897783), np.float64(2825.045899438576))

In [31]:
import metpy
from metpy.constants import Lv, Cp_d
from metpy.units import units
from metpy.calc import equivalent_potential_temperature
from tqdm import tqdm


surface_pressure = df["surface_PRES"].values * units.Pa
surface_dpt      = df["2m_above_ground_DPT"].values * units.K
surface_temp     = df["2m_above_ground_TMP"].values * units.K

theta_es = equivalent_potential_temperature(surface_pressure, surface_temp, surface_dpt)

# theta_es = []
# for sp, temp, dp in tqdm(zip(surface_pressure, surface_dpt, surface_temp), total=len(df)):
#     theta_es.append(equivalent_potential_temperature(
#         sp             * units.Pa, 
#         temp           * units.K,
#         dp             * units.K
#     ))

/playpen-ssd/levi/miniconda3/envs/wfc/lib/python3.14/site-packages/metpy/calc/thermo.py:1633: RuntimeWarning: invalid value encountered in power
  * (mpconsts.nounit.T0 / temperature) ** heat_power
/tmp/ipykernel_1994926/2745356401.py:12: UserWarning: Saturation mixing ratio is undefined for some requested pressure/temperature combinations. Total pressure must be greater than the water vapor saturation pressure for liquid water to be in equilibrium.
  theta_es = equivalent_potential_temperature(surface_pressure, surface_temp, surface_dpt)
/playpen-ssd/levi/miniconda3/envs/wfc/lib/python3.14/site-packages/metpy/calc/thermo.py:1943: RuntimeWarning: invalid value encountered in log
  t_l = 56 + 1. / (1. / (td - 56) + np.log(t / td) / 800.)
/playpen-ssd/levi/miniconda3/envs/wfc/lib/python3.14/site-packages/pint/facets/plain/quantity.py:1296: RuntimeWarning: invalid value encountered in power
  magnitude = new_self._magnitude**exponent


In [41]:
import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units
from tqdm import tqdm

tqdm.pandas()


"""
Calculate the lowest 100mb mean mixing ratio and add it as a column to a dataframe.

Requirements:
    pip install metpy pandas numpy
"""

import pandas as pd
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units


def calculate_lowest_100mb_mean_mixing_ratio(row):
    """
    Calculate the mean mixing ratio in the lowest 100 mb layer.
    
    Parameters
    ----------
    row : pd.Series
        A row from the dataframe containing surface_PRES, 2m_above_ground_DPT,
        925mb_DPT, and 850mb_DPT
    
    Returns
    -------
    float
        Mean mixing ratio in g/kg
    """
    # Surface pressure in Pa -> hPa
    sfc_pres_hpa = float(row['surface_PRES']) / 100.0
    top_pres_hpa = sfc_pres_hpa - 100.0
    
    # Collect levels within the lowest 100 mb
    levels = []
    
    # Surface (2m)
    levels.append((sfc_pres_hpa, float(row['2m_above_ground_DPT'])))
    
    # 925 mb (if within range)
    if 925 >= top_pres_hpa and 925 <= sfc_pres_hpa:
        levels.append((925.0, float(row['925mb_DPT'])))
    
    # 850 mb (if within range)
    if 850 >= top_pres_hpa and 850 <= sfc_pres_hpa:
        levels.append((850.0, float(row['850mb_DPT'])))
    
    mixing_ratios = []
    for pres_hpa, dewpoint_K in levels:
        pressure = pres_hpa * units.hPa
        dewpoint = dewpoint_K * units.K
        
        # Saturation vapor pressure at the dewpoint = actual vapor pressure
        vapor_pressure = mpcalc.saturation_vapor_pressure(dewpoint)
        
        # Mixing ratio from vapor pressure and total pressure
        w = mpcalc.mixing_ratio(vapor_pressure, pressure)
        
        # Extract magnitude and ensure it's a real float
        mixing_ratios.append(float(np.real(w.to('g/kg').magnitude)))
    
    return np.mean(mixing_ratios)


def add_mixing_ratio_column(df):
    """
    Add the 100_mb_mean_mixing_ratio column to a dataframe.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame containing required columns: surface_PRES, 2m_above_ground_DPT,
        925mb_DPT, 850mb_DPT
    
    Returns
    -------
    pd.DataFrame
        DataFrame with new '100_mb_mean_mixing_ratio' column added
    """
    df['100_mb_mean_mixing_ratio'] = df.apply(calculate_lowest_100mb_mean_mixing_ratio, axis=1)
    return df



df['100_mb_mean_mixing_ratio'] = df.progress_apply(calculate_lowest_100mb_mean_mixing_ratio, axis=1)

100%|██████████| 1557506/1557506 [18:09<00:00, 1429.28it/s]


In [91]:
df_nnan = df.dropna()
df_nnan = df_nnan[(df_nnan["2m_above_ground_DPT"] > 0) & (df_nnan["925mb_DPT"] > 0) & (df_nnan["850mb_DPT"] > 0) & (df_nnan["700mb_DPT"] > 0) & (df_nnan["500mb_DPT"] > 0) & (df_nnan["2m_above_ground_DPT"] > 0)]
df_nnan

,gauge_idx,start_datetime_utc,end_datetime_utc,gauge_acc_in,mrms_q3evap_qpe,lat,lon,700mb_UGRD,850mb_DPT,850mb_TMP,...,LCL_LFC_RH,0-3km_RH,0-5km_RH,DCAPE,low_level_lapse_rate,3hr_lapse_rate_change,sfc_850_pw,sfc_700_pw,850_700_mean_wind,CWCD
12,4564.0,2021-01-23 23:06:00+00:00,2021-01-24 00:06:00+00:00,0.0,0.000000,36.028250,-115.003639,3.759766,268.250000,276.750000,...,29.758370,35.305114,21.909930,155.279058,8.794788,0.0,2.739051,6.916189,3.850282,-85.000000
13,4779.0,2021-01-23 23:06:00+00:00,2021-01-24 00:06:00+00:00,0.0,0.000000,36.021861,-114.958750,3.884766,268.500000,276.750000,...,21.476085,30.685404,18.030886,158.431926,8.794788,0.0,2.890112,7.008753,3.878997,-123.000000
14,4724.0,2021-01-23 23:06:00+00:00,2021-01-24 00:06:00+00:00,0.0,0.015748,35.939417,-115.077750,2.884766,266.000000,277.250000,...,16.709872,20.972270,18.947291,131.984436,9.120521,0.0,1.355027,5.083167,3.371732,2.000000
15,4709.0,2021-01-23 23:06:00+00:00,2021-01-24 00:06:00+00:00,0.0,0.007874,35.925000,-115.117000,2.822266,265.750000,277.250000,...,13.553750,18.129263,17.823468,139.777327,9.114583,0.0,1.659570,5.281450,3.437455,-78.000000
16,4564.0,2021-01-23 23:08:00+00:00,2021-01-24 00:08:00+00:00,0.0,0.000000,36.028250,-115.003639,3.759766,268.250000,276.750000,...,29.758370,35.305114,21.909930,155.279058,8.794788,0.0,2.739051,6.916189,3.850282,-85.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1557501,3229.0,2025-07-20 03:40:00+00:00,2025-07-20 04:40:00+00:00,0.0,0.007874,36.802333,-114.953472,2.692192,278.963867,300.408142,...,47.478949,52.931341,20.004539,159.611884,8.575298,0.0,4.993778,13.611620,5.579225,1268.199219
1557502,3229.0,2025-07-20 03:42:00+00:00,2025-07-20 04:42:00+00:00,0.0,0.007874,36.802333,-114.953472,2.692192,278.963867,300.408142,...,47.478949,52.931341,20.004539,159.611884,8.575298,0.0,4.993778,13.611620,5.579225,1268.199219
1557503,3229.0,2025-07-20 03:44:00+00:00,2025-07-20 04:44:00+00:00,0.0,0.003937,36.802333,-114.953472,2.692192,278.963867,300.408142,...,47.478949,52.931341,20.004539,159.611884,8.575298,0.0,4.993778,13.611620,5.579225,1268.199219
1557504,3229.0,2025-07-20 03:46:00+00:00,2025-07-20 04:46:00+00:00,0.0,0.003937,36.802333,-114.953472,2.692192,278.963867,300.408142,...,47.478949,52.931341,20.004539,159.611884,8.575298,0.0,4.993778,13.611620,5.579225,1268.199219


In [92]:
df_nnan.to_csv("data/2021-01-01_2025-07-25_hrrr_env_final_v3.csv", index=False)

In [ ]:
df_nnan["lowest_100mb_mean_mixing_ratio"].max(), df_nnan["925mb_700mb_DPT"].min()

(np.float64(289.875), np.float64(247.29166666666663))

In [101]:
df_nnan[df_nnan["LCL_LFC_RH"] == 100.00]

,gauge_idx,start_datetime_utc,end_datetime_utc,gauge_acc_in,mrms_q3evap_qpe,lat,lon,700mb_UGRD,850mb_DPT,850mb_TMP,...,LCL_LFC_RH,0-3km_RH,0-5km_RH,DCAPE,low_level_lapse_rate,3hr_lapse_rate_change,sfc_850_pw,sfc_700_pw,850_700_mean_wind,CWCD
96577,4314.0,2021-03-13 16:00:00+00:00,2021-03-13 17:00:00+00:00,0.00,0.0,36.107511,-115.397828,2.990234,257.750000,275.500000,...,100.0,80.971356,39.906332,108.302514,7.501631,0.326158,0.251652,2.360049,9.478656,-496.000000
96578,4314.0,2021-03-13 16:02:00+00:00,2021-03-13 17:02:00+00:00,0.00,0.0,36.107511,-115.397828,2.990234,257.750000,275.500000,...,100.0,80.971356,39.906332,108.302514,7.501631,0.326158,0.251652,2.360049,9.478656,-496.000000
96579,4314.0,2021-03-13 16:04:00+00:00,2021-03-13 17:04:00+00:00,0.00,0.0,36.107511,-115.397828,2.990234,257.750000,275.500000,...,100.0,80.971356,39.906332,108.302514,7.501631,0.326158,0.251652,2.360049,9.478656,-496.000000
96580,4314.0,2021-03-13 16:06:00+00:00,2021-03-13 17:06:00+00:00,0.00,0.0,36.107511,-115.397828,2.990234,257.750000,275.500000,...,100.0,80.971356,39.906332,108.302514,7.501631,0.000000,0.251652,2.360049,9.478656,-496.000000
164755,4794.0,2021-12-15 18:58:00+00:00,2021-12-15 19:58:00+00:00,0.00,0.0,36.033194,-114.932556,5.003906,254.375000,272.250000,...,100.0,84.224468,23.148113,162.697593,5.095332,-0.275762,1.915582,3.217635,6.587951,-332.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1318030,3834.0,2025-02-15 19:50:00+00:00,2025-02-15 20:50:00+00:00,0.12,0.0,36.249472,-115.630194,-0.450741,256.747925,279.183319,...,100.0,82.051462,23.136004,27.819466,4.866236,0.000000,0.000000,0.455659,8.994071,-279.391602
1318031,3834.0,2025-02-15 19:52:00+00:00,2025-02-15 20:52:00+00:00,0.12,0.0,36.249472,-115.630194,-0.450741,256.747925,279.183319,...,100.0,82.051462,23.136004,27.819466,4.866236,0.000000,0.000000,0.455659,8.994071,-279.391602
1318032,3834.0,2025-02-15 19:54:00+00:00,2025-02-15 20:54:00+00:00,0.12,0.0,36.249472,-115.630194,-0.450741,256.747925,279.183319,...,100.0,82.051462,23.136004,27.819466,4.866236,0.000000,0.000000,0.455659,8.994071,-279.391602
1318033,3834.0,2025-02-15 19:56:00+00:00,2025-02-15 20:56:00+00:00,0.12,0.0,36.249472,-115.630194,-0.450741,256.747925,279.183319,...,100.0,82.051462,23.136004,27.819466,4.866236,0.000000,0.000000,0.455659,8.994071,-279.391602


In [47]:
df.to_csv("2021-01-01_2025-07-25_hrrr_env_final_v4", index=False)